# Rossmann: Random Forest (Stage1 - Prepare)

## Intro

Keggle competition: [Rossmann Store Sales](https://www.kaggle.com/c/rossmann-store-sales)

## Usage

**Input parameters**
1. RAW_TRAIN_CSV: Raw train data about sales
1. RAW_STORE_CSV:Raw train data about stores
1. PROCESSED_TRAIN_CSV:The name of the output file used to store the processed train data

**Output**
1. A file with processed train data ready to be used for model training


## Setup env

### Set global variables

In [1]:
!pwd

/opt/shared/notebooks


In [2]:
DATASETS_DIR = '../data'

In [3]:
# this cell is tagged `parameters`
RAW_TRAIN_CSV = DATASETS_DIR + '/raw/train.csv'
RAW_STORE_CSV = DATASETS_DIR + '/raw/store.csv'
PROCESSED_TRAIN_CSV = DATASETS_DIR + '/processed/tst-train.csv'

###  Install required packages

Se il notebook è eseguito su una macchina pulita installare i pacchetti necessari con i seguenti comandi ...

In [4]:
#!curl https://raw.githubusercontent.com/andrea-gioia/boostrap.ai/master/fastai07colab	 | bash

In [5]:
#!pip list

Se il notebook è eseguito all'interno di un ambiente virtuale conda con tutti i pacchetti specificati nel file di requirements già installati fare solo un check eseguendo i seguenti comandi ...

In [6]:
!conda env list

# conda environments:
#
base                     /opt/conda
custom                *  /opt/conda/envs/custom



In [7]:
!python -V

Python 3.7.3


In [8]:
#!conda list

### Import packagest

In [9]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [10]:
from fastai.imports import *
from fastai07.structured import *

#todo upgrade code to v1
#from fastai.tabular import *

#from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

ModuleNotFoundError: No module named 'fastai07'

## Stage 1: prepare

In [11]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

### Load raw dataset

In [12]:
df_raw = pd.read_csv(RAW_TRAIN_CSV, low_memory=False, parse_dates=["Date"])

In [13]:
display_all(df_raw.tail().T)

,1017204,1017205,1017206,1017207,1017208
Store,1111,1112,1113,1114,1115
DayOfWeek,2,2,2,2,2
Date,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00
Sales,0,0,0,0,0
Customers,0,0,0,0,0
Open,0,0,0,0,0
Promo,0,0,0,0,0
StateHoliday,a,a,a,a,a
SchoolHoliday,1,1,1,1,1


In [14]:
display_all(df_raw.describe(include='all').T)

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
Store,1.01721e+06,NaN,NaN,NaN,NaN,NaN,558.43,321.909,1,280,558,838,1115
DayOfWeek,1.01721e+06,NaN,NaN,NaN,NaN,NaN,3.99834,1.99739,1,2,4,6,7
Date,1017209,942,2015-06-09 00:00:00,1115,2013-01-01 00:00:00,2015-07-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sales,1.01721e+06,NaN,NaN,NaN,NaN,NaN,5773.82,3849.93,0,3727,5744,7856,41551
Customers,1.01721e+06,NaN,NaN,NaN,NaN,NaN,633.146,464.412,0,405,609,837,7388
Open,1.01721e+06,NaN,NaN,NaN,NaN,NaN,0.830107,0.375539,0,1,1,1,1
Promo,1.01721e+06,NaN,NaN,NaN,NaN,NaN,0.381515,0.485759,0,0,0,1,1
StateHoliday,1017209,4,0,986159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SchoolHoliday,1.01721e+06,NaN,NaN,NaN,NaN,NaN,0.178647,0.383056,0,0,0,0,1


### Load store dataset

In [15]:
df_store = pd.read_csv(RAW_STORE_CSV, low_memory=False)

In [16]:
display_all(df_store.tail().T)

,1110,1111,1112,1113,1114
Store,1111,1112,1113,1114,1115
StoreType,a,c,a,a,d
Assortment,a,c,c,c,c
CompetitionDistance,1900,1880,9260,870,5350
CompetitionOpenSinceMonth,6,4,NaN,NaN,NaN
CompetitionOpenSinceYear,2014,2006,NaN,NaN,NaN
Promo2,1,0,0,0,1
Promo2SinceWeek,31,NaN,NaN,NaN,22
Promo2SinceYear,2013,NaN,NaN,NaN,2012
PromoInterval,"Jan,Apr,Jul,Oct",NaN,NaN,NaN,"Mar,Jun,Sept,Dec"


In [17]:
display_all(df_store.describe(include='all').T)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Store,1115,NaN,NaN,NaN,558,322.017,1,279.5,558,836.5,1115
StoreType,1115,4,a,602,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Assortment,1115,3,a,593,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CompetitionDistance,1112,NaN,NaN,NaN,5404.9,7663.17,20,717.5,2325,6882.5,75860
CompetitionOpenSinceMonth,761,NaN,NaN,NaN,7.2247,3.21235,1,4,8,10,12
CompetitionOpenSinceYear,761,NaN,NaN,NaN,2008.67,6.19598,1900,2006,2010,2013,2015
Promo2,1115,NaN,NaN,NaN,0.512108,0.500078,0,0,1,1,1
Promo2SinceWeek,571,NaN,NaN,NaN,23.5954,14.142,1,13,22,37,50
Promo2SinceYear,571,NaN,NaN,NaN,2011.76,1.67494,2009,2011,2012,2013,2015
PromoInterval,571,3,"Jan,Apr,Jul,Oct",335,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Marge datasets (train+store)

In [18]:
df_raw = pd.merge(df_raw, df_store, on='Store')

In [19]:
display_all(df_raw.tail().T)

,1017204,1017205,1017206,1017207,1017208
Store,1115,1115,1115,1115,1115
DayOfWeek,6,5,4,3,2
Date,2013-01-05 00:00:00,2013-01-04 00:00:00,2013-01-03 00:00:00,2013-01-02 00:00:00,2013-01-01 00:00:00
Sales,4771,4540,4297,3697,0
Customers,339,326,300,305,0
Open,1,1,1,1,0
Promo,0,0,0,0,0
StateHoliday,0,0,0,0,a
SchoolHoliday,1,1,1,1,1
StoreType,d,d,d,d,d


## Feature engineering

### Expand dates

In [20]:
#df_raw = df_raw.sort_values(['Date'])
dates = df_raw['Date']
add_datepart(df_raw, 'Date', drop=True)

In [21]:
display_all(df_raw.tail().T)

,1017204,1017205,1017206,1017207,1017208
Store,1115,1115,1115,1115,1115
DayOfWeek,6,5,4,3,2
Sales,4771,4540,4297,3697,0
Customers,339,326,300,305,0
Open,1,1,1,1,0
Promo,0,0,0,0,0
StateHoliday,0,0,0,0,a
SchoolHoliday,1,1,1,1,1
StoreType,d,d,d,d,d
Assortment,c,c,c,c,c


In [22]:
df_raw.Year.head()

0    2015
1    2015
2    2015
3    2015
4    2015
Name: Year, dtype: int64

### Convert cotegorical features

In [23]:
train_cats(df_raw)

In [24]:
display_all(df_raw.describe(include='all').T)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Store,1.01721e+06,NaN,NaN,NaN,558.43,321.909,1,280,558,838,1115
DayOfWeek,1.01721e+06,NaN,NaN,NaN,3.99834,1.99739,1,2,4,6,7
Sales,1.01721e+06,NaN,NaN,NaN,5773.82,3849.93,0,3727,5744,7856,41551
Customers,1.01721e+06,NaN,NaN,NaN,633.146,464.412,0,405,609,837,7388
Open,1.01721e+06,NaN,NaN,NaN,0.830107,0.375539,0,1,1,1,1
Promo,1.01721e+06,NaN,NaN,NaN,0.381515,0.485759,0,0,0,1,1
StateHoliday,1017209,4,0,986159,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SchoolHoliday,1.01721e+06,NaN,NaN,NaN,0.178647,0.383056,0,0,0,0,1
StoreType,1017209,4,a,551627,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Assortment,1017209,3,a,537445,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Sembra che le categorie siano già correttamente ordinata ...

In [25]:
df_raw.StoreType.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [26]:
df_raw.Assortment.cat.categories

Index(['a', 'b', 'c'], dtype='object')

...  non dobbiamo pertanto  procedere al riordino


```
df_raw.StoreType.cat.set_categories(['a', 'b', 'c', 'd'], ordered=True, inplace=True)
```



In [27]:
df_raw.StoreType.cat.codes[:5]

0    2
1    2
2    2
3    2
4    2
dtype: int8

### Handle missing values

In [28]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

Assortment                   0.000000
CompetitionDistance          0.002597
CompetitionOpenSinceMonth    0.317878
CompetitionOpenSinceYear     0.317878
Customers                    0.000000
Day                          0.000000
DayOfWeek                    0.000000
Dayofweek                    0.000000
Dayofyear                    0.000000
Elapsed                      0.000000
Is_month_end                 0.000000
Is_month_start               0.000000
Is_quarter_end               0.000000
Is_quarter_start             0.000000
Is_year_end                  0.000000
Is_year_start                0.000000
Month                        0.000000
Open                         0.000000
Promo                        0.000000
Promo2                       0.000000
Promo2SinceWeek              0.499436
Promo2SinceYear              0.499436
PromoInterval                0.499436
Sales                        0.000000
SchoolHoliday                0.000000
StateHoliday                 0.000000
Store       

In [29]:
df_raw.shape

(1017209, 30)

We'll replace categories with their numeric codes, handle missing continuous values, and split the dependent variable into a separate variable.

In [30]:
X_train, y_train, nas = proc_df(df_raw, 'Sales')

In [31]:
display_all(X_train.isnull().sum().sort_index()/len(X_train))

Assortment                      0.0
CompetitionDistance             0.0
CompetitionDistance_na          0.0
CompetitionOpenSinceMonth       0.0
CompetitionOpenSinceMonth_na    0.0
CompetitionOpenSinceYear        0.0
CompetitionOpenSinceYear_na     0.0
Customers                       0.0
Day                             0.0
DayOfWeek                       0.0
Dayofweek                       0.0
Dayofyear                       0.0
Elapsed                         0.0
Is_month_end                    0.0
Is_month_start                  0.0
Is_quarter_end                  0.0
Is_quarter_start                0.0
Is_year_end                     0.0
Is_year_start                   0.0
Month                           0.0
Open                            0.0
Promo                           0.0
Promo2                          0.0
Promo2SinceWeek                 0.0
Promo2SinceWeek_na              0.0
Promo2SinceYear                 0.0
Promo2SinceYear_na              0.0
PromoInterval               

In [32]:
df_raw['Sales'][-5:]

1017204    4771
1017205    4540
1017206    4297
1017207    3697
1017208       0
Name: Sales, dtype: int64

In [33]:
y_train[-5:]

array([4771, 4540, 4297, 3697,    0])

### Save prepared data

In [34]:
X_train.shape, y_train.shape

((1017209, 34), (1017209,))

In [35]:
display_all(df_raw.tail().T)

,1017204,1017205,1017206,1017207,1017208
Store,1115,1115,1115,1115,1115
DayOfWeek,6,5,4,3,2
Sales,4771,4540,4297,3697,0
Customers,339,326,300,305,0
Open,1,1,1,1,0
Promo,0,0,0,0,0
StateHoliday,0,0,0,0,a
SchoolHoliday,1,1,1,1,1
StoreType,d,d,d,d,d
Assortment,c,c,c,c,c


In [36]:
y_train = pd.DataFrame(y_train,columns=["Sales"])
y_train[-5:]

,Sales
1017204,4771
1017205,4540
1017206,4297
1017207,3697
1017208,0


In [37]:
df_processed_train = pd.concat([X_train, y_train], axis = 1);
df_processed_train = pd.concat([df_processed_train, dates], axis = 1);
df_processed_train.shape, display_all(df_processed_train.tail().T)

,1017204,1017205,1017206,1017207,1017208
Store,1115,1115,1115,1115,1115
DayOfWeek,6,5,4,3,2
Customers,339,326,300,305,0
Open,1,1,1,1,0
Promo,0,0,0,0,0
StateHoliday,1,1,1,1,2
SchoolHoliday,1,1,1,1,1
StoreType,4,4,4,4,4
Assortment,3,3,3,3,3
CompetitionDistance,5350,5350,5350,5350,5350


((1017209, 36), None)

In [38]:
nas

{'CompetitionDistance': 2330.0,
 'CompetitionOpenSinceMonth': 8.0,
 'CompetitionOpenSinceYear': 2010.0,
 'Promo2SinceWeek': 22.0,
 'Promo2SinceYear': 2012.0}

In [39]:
df_processed_train = df_processed_train.sort_values(['Date'])
display_all(df_processed_train.tail().T)

,557530,212332,982701,769288,0
Store,612,235,1078,845,1
DayOfWeek,5,5,5,5,5
Customers,493,548,646,378,555
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,1,1,1,1,1
SchoolHoliday,0,1,1,1,1
StoreType,4,1,4,4,3
Assortment,3,1,3,1,1
CompetitionDistance,2490,5710,670,7860,1270


In [40]:
df_processed_train.to_csv(PROCESSED_TRAIN_CSV)

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/tst-train.csv'

In [ ]:
# FINE